In [1]:
!pip install datasets
!pip install torch torchvision torchaudio
!pip install transformers
!pip install rouge-score nltk
!pip install sentencepiece
!pip install evaluate

  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 0.19.4 which is incompatible.
You should consider upgrading via the 'C:\Users\emely\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\emely\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


  Using cached huggingface_hub-0.17.3-py3-none-any.whl (295 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.4
    Uninstalling huggingface-hub-0.19.4:
      Successfully uninstalled huggingface-hub-0.19.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.15.0 requires huggingface-hub>=0.18.0, but you have huggingface-hub 0.17.3 which is incompatible.
You should consider upgrading via the 'C:\Users\emely\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24990 sha256=f441c3d1a15de8fb102406d6a8b55055c6775951548d58ebd58784890110f1de
  Stored in directory: c:\users\emely\appdata\local\pip\cache\wheels\5f\dd\89\461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


You should consider upgrading via the 'C:\Users\emely\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\emely\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


  Using cached huggingface_hub-0.19.4-py3-none-any.whl (311 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 0.19.4 which is incompatible.
You should consider upgrading via the 'C:\Users\emely\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Напоминаю задание:
1. Взять модель для суммаризации текстов https://huggingface.co/IlyaGusev/mbart_ru_sum_gazeta и запустить суммаризацию текстов на тестовой части вот этого датасета: https://huggingface.co/datasets/IlyaGusev/gazeta. Код, который это делает, есть в карточке модели. Посчитать метрики blue и rouge
2. Дообучить модель google/mt5-small для суммаризации текстов из датасета https://huggingface.co/datasets/IlyaGusev/gazeta, запустить суммаризацию на тестовой части датасета, посчитать метрики blue и rouge
Таким образом, у вас должно получиться сравнение метрик для оценки качества суммаризации, сделанной двумя разными моделями

# Испорты и загрузка дата сета

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\emely\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from datasets import load_dataset, load_metric
from transformers import (
    DataCollatorForSeq2Seq,
    MT5ForConditionalGeneration,
    MT5Tokenizer,
    pipeline,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
) 
import evaluate
import numpy as np
import datasets

# Загрузка дата сета

In [4]:
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")  # cointegrated/rut5-small google/mt5-small
model = MT5ForConditionalGeneration.from_pretrained('google/mt5-small')

# tokenizer = MT5Tokenizer.from_pretrained("results/checkpoint-121500")  # это если мы хотим подгрузить веса модели которую уже обучали
# model = MT5ForConditionalGeneration.from_pretrained('results/checkpoint-121500')

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

c:\Users\emely\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\emely\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Обработка данных лата сета

 - max_target_tokens_count и max_source_tokens_count - это максимальное количество токенов (слов или подобных единиц) для целевого текста (labels) и исходного текста (text).
 - preprocess_function - это функция предварительной обработки данных, которая принимает на вход словарь examples. Этот словарь, вероятно, содержит текст (text) и цель (summary), которые представляют собой данные, подготовленные для обучения модели.

In [5]:
max_target_tokens_count = 20
max_source_tokens_count = 1024

def preprocess_function(examples):
    inputs = examples["text"]
    model_inputs = tokenizer(inputs, max_length=max_source_tokens_count, truncation=True)
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_tokens_count, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [6]:
dataset = load_dataset('IlyaGusev/gazeta',revision="v2.0")
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60964 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6793 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6369 [00:00<?, ? examples/s]

Map:   0%|          | 0/60964 [00:00<?, ? examples/s]

Map:   0%|          | 0/6793 [00:00<?, ? examples/s]

Map:   0%|          | 0/6369 [00:00<?, ? examples/s]

In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [13]:
rouge = load_metric("rouge")
bleu = evaluate.load("bleu")

<ipython-input-13-9921d770c86d>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


In [14]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = {}
    result_rouge = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    # Extract a few results
    result.update({key: value.mid.fmeasure * 100 for key, value in result_rouge.items()})
    
    result_bleu = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    # Extract a few results
    result["bleu"] = result_bleu["bleu"] * 100
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    result["char_len"] = np.mean([len(t) for t in decoded_preds])
    
    return {k: round(v, 4) for k, v in result.items()}

In [15]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/NLP/results",
    evaluation_strategy="steps",
    eval_steps=25,
    logging_steps=25,
    learning_rate=4e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=False,
    predict_with_generate=True,
    generation_max_length=max_target_tokens_count,
    generation_num_beams=5
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()